In [1]:
import geopandas as gpd

ws = gpd.read_file("../data/Geometry/WatershedGeomCAMELS.gpkg")
ws.set_index("gage_id", inplace=True)

gauge = gpd.read_file("../data/Geometry/GaugeGeomCAMELS.gpkg")
gauge.set_index("gage_id", inplace=True)

### Perform clustering based on static attributes

In [ ]:
static_data = 

### Perform clustering based on hydrological attributes